In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
#income per household per municipality
income = pd.read_csv('2019_5Y_Income.csv',skiprows=1)
#occupation of people in municipalities
occupation = pd.read_csv('2019_5Y_Occupation.csv',skiprows=1)
#educational attainments of people in a municipality
edu = pd.read_csv('2019_5Y_EducationalAttainment.csv',skiprows=1)
#data of evictions
cdf = pd.read_csv('Eviction Filings thru July2021.csv')
#Munis that had high eviction_count to population ratio 
munis_to_focus = ['Fall River', 'Worcester', 'New Bedford', 'Fitchburg', 'Gardner', 'Webster', 
                  'Framingham', 'Rowe', 'North Adams', 'Marlborough']

In [3]:
counts = cdf.muni.value_counts().reset_index()
counts.rename(columns={'index':'Muni','muni':'Eviction Numbers'},inplace=True)

In [4]:
#data cleaning function 1
def df_clean(df):
    muni_list = []
    for x in range(df.shape[0]):
        wsplit = []
        wsplit = df.iloc[x]['Geographic Area Name'].split()
        word = ' '
        reject_list = ['town','Town','city','City','city,','City,','town,','Town,']
        for w in wsplit:
            if w in reject_list:
                break
            else:    
                word += ' ' + w 
        muni_list.append(word)    
    df['Geographic Area Name'] = muni_list
    df['Geographic Area Name'] = df['Geographic Area Name'].str.lstrip()
    df['Geographic Area Name'] = df['Geographic Area Name'].str.rstrip()    
    
    return df

In [5]:
#data cleaning function 2
def keep_estimates(idf):
    boolCols = idf.columns.str.contains('Estimate')
    newCols = list(idf.columns[0:2])
    for col in range(2,len(boolCols)):
        if boolCols[col] == True:
            newCols.append(idf.columns[col])
    df = idf[newCols]
    
    return df

In [6]:
muni_df = counts[counts.Muni.isin(munis_to_focus)]

In [7]:
income = df_clean(keep_estimates(income));
occupation = df_clean(keep_estimates(occupation));
edu = df_clean(keep_estimates(edu));

In [8]:
#Merging munis with corresponding socio-economic features
income_df =  pd.merge(muni_df,income,left_on='Muni',right_on='Geographic Area Name').drop(columns=['id','Geographic Area Name'])
occu_df =  pd.merge(muni_df,occupation,left_on='Muni',right_on='Geographic Area Name').drop(columns=['id','Geographic Area Name'])
edu_df =  pd.merge(muni_df,edu,left_on='Muni',right_on='Geographic Area Name').drop(columns=['id','Geographic Area Name'])

In [9]:
print(income_df)

          Muni  Eviction Numbers  Estimate!!Households!!Total  \
0    Worcester               576                        71595   
1   Fall River               354                        38456   
2  New Bedford               293                        38888   
3   Framingham               169                        28222   
4    Fitchburg               121                        14965   
5  Marlborough                83                        15730   
6      Gardner                51                         8228   
7      Webster                41                         6946   
8  North Adams                29                         5568   
9         Rowe                 1                          191   

  Estimate!!Households!!Total!!Less than $10,000  \
0                                           10.4   
1                                            8.4   
2                                            8.8   
3                                            5.1   
4                       

In [10]:
print(occu_df)

          Muni  Eviction Numbers  \
0    Worcester               576   
1   Fall River               354   
2  New Bedford               293   
3   Framingham               169   
4    Fitchburg               121   
5  Marlborough                83   
6      Gardner                51   
7      Webster                41   
8  North Adams                29   
9         Rowe                 1   

   Estimate!!Total!!Civilian employed population 16 years and over  \
0                                              87722                 
1                                              39484                 
2                                              44075                 
3                                              41197                 
4                                              19626                 
5                                              23134                 
6                                               9868                 
7                                          

In [11]:
print(edu_df)

          Muni  Eviction Numbers  \
0    Worcester               576   
1   Fall River               354   
2  New Bedford               293   
3   Framingham               169   
4    Fitchburg               121   
5  Marlborough                83   
6      Gardner                51   
7      Webster                41   
8  North Adams                29   
9         Rowe                 1   

   Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years  \
0                                              28769                           
1                                               7924                           
2                                               9674                           
3                                               6401                           
4                                               5284                           
5                                               3179                           
6                                          